In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import stan
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import cm
from scipy import stats
from scipy.stats import pearsonr
import pickle

FONT_SIZE = 8

plt.rc('font', size=FONT_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=FONT_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=FONT_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=FONT_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=FONT_SIZE)    # legend fontsize
plt.rc('figure', titlesize=FONT_SIZE)  # fontsize of the figure title
plt.rcParams['pdf.fonttype'] = 42

In [2]:
# from IPython import get_ipython
# get_ipython().run_line_magic('matplotlib', 'qt')
# plt.style.use('seaborn-white')

#%%

### load data ###
dat_RC = pd.read_csv('smFISH_RC.csv')

jtime_RC = dat_RC['Time'].values
AreaNormed_RC = dat_RC['AreaNormed'].values
CountsNr1d1_RC = dat_RC['Counts Nr1d1'].values
CountsCry1_RC = dat_RC['Counts Cry1'].values

dat_BC = pd.read_csv('smFISH_BC.csv')

jtime_BC = dat_BC['Time'].values
AreaNormed_BC = dat_BC['AreaNormed'].values
CountsBmal1_BC = dat_BC['Counts Bmal1'].values
CountsCry1_BC = dat_BC['Counts Cry1'].values

CountsCry1TOT = np.concatenate((CountsCry1_BC,CountsCry1_RC))
jtime_Cry1TOT = np.concatenate((jtime_BC,jtime_RC))
AreaCry1TOT = np.concatenate((AreaNormed_BC,AreaNormed_RC))

w = 2 * np.pi / 24 

TimevecRC = np.array([21.,25.,29.,33.,37.,41.]).reshape(-1,1)
TimevecBC = np.array([17.,21.,25.,29.,33.,37.,41.]).reshape(-1,1)

In [3]:
import asyncio
import nest_asyncio
nest_asyncio.apply()
asyncio.run(asyncio.sleep(1))

In [7]:
#%%
# load Fourier parameters
Nr1d1_params, Cry1_params, Bmal1_params = pickle.load(open('FourierParams.pkl', 'rb'))

In [4]:
#%%

# MODEL 1

model1 = """
data{
    int<lower=1> N1;
    array[N1] int CountsNr1d1_RC;
    array[N1] int CountsCry1_RC; 
    
    array[N1] real jtime_RC;
    array[N1] real AreaNormed_RC;
    int<lower=1> N2;
    array[N2] int CountsBmal1_BC;
    array[N2] int CountsCry1_BC;
    array[N2] real jtime_BC;
    array[N2] real AreaNormed_BC;
    int <lower=1> N_f;
    array[N_f] real Nr1d1_params;
    array[N_f] real Cry1_params;
    array[N_f] real Bmal1_params;
    real w;
}
parameters{
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1;
        b2 = burstNr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1;
        b4 = burstBmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    freq_scaleCry1~ normal(0,100);
    freq_scaleNr1d1~ normal(0,100);
    freq_scaleBmal1~ normal(0,100);
    burstCry1~ normal(0,100);
    burstNr1d1~ normal(0,100);
    burstBmal1~ normal(0,100);
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1;
        b2 = burstNr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1;
        b4 = burstBmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}
 

"""

#%%


dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [5]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model1, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=16, num_samples=2000)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_i5tl3agu/model_vxfqtzda.stan', line 74, column 25: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_i5tl3agu/model_vxfqtzda.stan', line 73, column 25: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_i5tl3agu/model_vxfqtzda.stan', line 72, column 24: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_i5tl3agu/model_vxfqtzda.stan', line 71, column 30: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_i5tl3agu/model_vxfqtzda.stan', line 7

Sampling:  42% (20400/48000)
Sampling:  43% (20500/48000)
Sampling:  43% (20600/48000)
Sampling:  43% (20700/48000)
Sampling:  43% (20800/48000)
Sampling:  44% (20900/48000)
Sampling:  44% (21000/48000)
Sampling:  44% (21100/48000)
Sampling:  44% (21200/48000)
Sampling:  44% (21300/48000)
Sampling:  45% (21400/48000)
Sampling:  45% (21500/48000)
Sampling:  45% (21600/48000)
Sampling:  45% (21700/48000)
Sampling:  45% (21800/48000)
Sampling:  46% (21900/48000)
Sampling:  46% (22000/48000)
Sampling:  46% (22100/48000)
Sampling:  46% (22200/48000)
Sampling:  46% (22300/48000)
Sampling:  47% (22400/48000)
Sampling:  47% (22500/48000)
Sampling:  47% (22600/48000)
Sampling:  47% (22700/48000)
Sampling:  48% (22800/48000)
Sampling:  48% (22900/48000)
Sampling:  48% (23000/48000)
Sampling:  48% (23100/48000)
Sampling:  48% (23200/48000)
Sampling:  49% (23300/48000)
Sampling:  49% (23400/48000)
Sampling:  49% (23500/48000)
Sampling:  49% (23600/48000)
Sampling:  49% (23700/48000)
Sampling:  50%

Sampling:  80% (38200/48000)
Sampling:  80% (38300/48000)
Sampling:  80% (38400/48000)
Sampling:  80% (38500/48000)
Sampling:  80% (38600/48000)
Sampling:  81% (38700/48000)
Sampling:  81% (38800/48000)
Sampling:  81% (38900/48000)
Sampling:  81% (39000/48000)
Sampling:  81% (39100/48000)
Sampling:  82% (39200/48000)
Sampling:  82% (39300/48000)
Sampling:  82% (39400/48000)
Sampling:  82% (39500/48000)
Sampling:  82% (39600/48000)
Sampling:  83% (39700/48000)
Sampling:  83% (39800/48000)
Sampling:  83% (39900/48000)
Sampling:  83% (40000/48000)
Sampling:  84% (40100/48000)
Sampling:  84% (40200/48000)
Sampling:  84% (40300/48000)
Sampling:  84% (40400/48000)
Sampling:  84% (40500/48000)
Sampling:  85% (40600/48000)
Sampling:  85% (40700/48000)
Sampling:  85% (40800/48000)
Sampling:  85% (40900/48000)
Sampling:  85% (41000/48000)
Sampling:  86% (41100/48000)
Sampling:  86% (41200/48000)
Sampling:  86% (41300/48000)
Sampling:  86% (41400/48000)
Sampling:  86% (41500/48000)
Sampling:  87%

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 0.003982 seconds
  1000 transitions using 10 leapfrog steps per transition would take 39.82 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_a0j6adhu/model_vxfqtzda.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_a0j6adhu/model_vxfqtzda.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan

...

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j6adhu/model_vxfqtzda.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j6adhu/model_vxfqtzda.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j6adhu/model_vxfqtzda.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_a0j6adhu/model_vxfqtzda.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j6adhu/model_vxfqtzda.stan', line 65, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_a0j

In [6]:
with open('model_1.pkl', 'wb') as f:
    pickle.dump(posterior, f)    

In [7]:
with open('fit_1.pkl', 'wb') as g:
    pickle.dump(fit, g)         

In [8]:
#%%

# MODEL 2

model1 = """
data{
    int<lower=1> N1;
    int CountsNr1d1_RC[N1];
    int CountsCry1_RC[N1];    
    real jtime_RC[N1];
    real AreaNormed_RC[N1];
    int<lower=1> N2;
    int CountsBmal1_BC[N2];
    int CountsCry1_BC[N2];    
    real jtime_BC[N2];
    real AreaNormed_BC[N2];
    int <lower=1> N_f;
    real Nr1d1_params[N_f];
    real Cry1_params[N_f];
    real Bmal1_params[N_f];
    real w;
}
parameters{
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    freq_scaleCry1~ normal(0,100);
    freq_scaleNr1d1~ normal(0,100);
    freq_scaleBmal1~ normal(0,100);
    burstCry1~ normal(0,100);
    burstNr1d1~ normal(0,100);
    burstBmal1~ normal(0,100);
    beta_v_Cry1~ normal(0,100);
    beta_v_Nr1d1~ normal(0,100);
    beta_v_Bmal1~ normal(0,100);
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}    

"""

In [9]:
#%%

dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [10]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model1, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=16, num_samples=2000)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_baq56h8_/model_saf2m3bt.stan', line 4, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_baq56h8_/model_saf2m3bt.stan', line 5, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_baq56h8_/model_saf2m3bt.stan', line 6, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format

Sampling:  11% (5100/48000)
Sampling:  11% (5200/48000)
Sampling:  11% (5300/48000)
Sampling:  11% (5400/48000)
Sampling:  12% (5600/48000)
Sampling:  12% (5700/48000)
Sampling:  12% (5800/48000)
Sampling:  12% (5900/48000)
Sampling:  12% (6000/48000)
Sampling:  13% (6100/48000)
Sampling:  13% (6200/48000)
Sampling:  13% (6300/48000)
Sampling:  13% (6400/48000)
Sampling:  14% (6500/48000)
Sampling:  14% (6600/48000)
Sampling:  14% (6700/48000)
Sampling:  14% (6800/48000)
Sampling:  14% (6900/48000)
Sampling:  15% (7100/48000)
Sampling:  15% (7200/48000)
Sampling:  15% (7300/48000)
Sampling:  16% (7500/48000)
Sampling:  16% (7700/48000)
Sampling:  16% (7900/48000)
Sampling:  17% (8100/48000)
Sampling:  17% (8200/48000)
Sampling:  17% (8300/48000)
Sampling:  18% (8400/48000)
Sampling:  18% (8600/48000)
Sampling:  18% (8700/48000)
Sampling:  18% (8800/48000)
Sampling:  19% (8900/48000)
Sampling:  19% (9000/48000)
Sampling:  19% (9100/48000)
Sampling:  19% (9300/48000)
Sampling:  20% (9400

Sampling:  54% (25900/48000)
Sampling:  54% (26000/48000)
Sampling:  54% (26100/48000)
Sampling:  55% (26200/48000)
Sampling:  55% (26300/48000)
Sampling:  55% (26400/48000)
Sampling:  55% (26500/48000)
Sampling:  55% (26600/48000)
Sampling:  56% (26700/48000)
Sampling:  56% (26800/48000)
Sampling:  56% (26900/48000)
Sampling:  56% (27000/48000)
Sampling:  56% (27100/48000)
Sampling:  57% (27200/48000)
Sampling:  57% (27300/48000)
Sampling:  57% (27400/48000)
Sampling:  57% (27500/48000)
Sampling:  58% (27600/48000)
Sampling:  58% (27700/48000)
Sampling:  58% (27800/48000)
Sampling:  58% (27900/48000)
Sampling:  58% (28000/48000)
Sampling:  59% (28100/48000)
Sampling:  59% (28200/48000)
Sampling:  59% (28300/48000)
Sampling:  59% (28400/48000)
Sampling:  59% (28500/48000)
Sampling:  60% (28600/48000)
Sampling:  60% (28700/48000)
Sampling:  60% (28800/48000)
Sampling:  60% (28900/48000)
Sampling:  60% (29000/48000)
Sampling:  61% (29100/48000)
Sampling:  61% (29200/48000)
Sampling:  61%

Sampling:  91% (43700/48000)
Sampling:  91% (43800/48000)
Sampling:  91% (43900/48000)
Sampling:  92% (44000/48000)
Sampling:  92% (44100/48000)
Sampling:  92% (44200/48000)
Sampling:  92% (44300/48000)
Sampling:  92% (44400/48000)
Sampling:  93% (44500/48000)
Sampling:  93% (44600/48000)
Sampling:  93% (44700/48000)
Sampling:  93% (44800/48000)
Sampling:  94% (44900/48000)
Sampling:  94% (45000/48000)
Sampling:  94% (45100/48000)
Sampling:  94% (45200/48000)
Sampling:  94% (45300/48000)
Sampling:  95% (45400/48000)
Sampling:  95% (45500/48000)
Sampling:  95% (45600/48000)
Sampling:  95% (45700/48000)
Sampling:  95% (45800/48000)
Sampling:  96% (45900/48000)
Sampling:  96% (46000/48000)
Sampling:  96% (46100/48000)
Sampling:  96% (46200/48000)
Sampling:  96% (46300/48000)
Sampling:  97% (46400/48000)
Sampling:  97% (46500/48000)
Sampling:  97% (46600/48000)
Sampling:  97% (46700/48000)
Sampling:  98% (46800/48000)
Sampling:  98% (46900/48000)
Sampling:  98% (47000/48000)
Sampling:  98%

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 70, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained varia

...

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is -nan, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained vari

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained var

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 68, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variabl

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is 0, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variabl

  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 70, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Gradient evaluation took 0.004449 seconds
  1000 transitions using 10 leapfrog steps per transition would take 44.49 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_9d6t1i_0/model_saf2m3bt.stan', line 67, column 4 to column 47)
  If this wa

In [11]:
with open('model_2.pkl', 'wb') as f:
    pickle.dump(posterior, f)
    
with open('fit_2.pkl', 'wb') as g:
    pickle.dump(fit, g)

In [12]:
#%%
# MODEL 3 - phase noise

model = """
data{
    int<lower=1> N1;
    int CountsNr1d1_RC[N1];
    int CountsCry1_RC[N1];    
    real jtime_RC[N1];
    real AreaNormed_RC[N1];
    int<lower=1> N2;
    int CountsBmal1_BC[N2];
    int CountsCry1_BC[N2];    
    real jtime_BC[N2];
    real AreaNormed_BC[N2];
    real kappa;
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
    int <lower=1> N_f;
    real Nr1d1_params[N_f];
    real Cry1_params[N_f];
    real Bmal1_params[N_f];
    real w;
}
parameters{
    real phi_i_add_RC[N1];
    real phi_i_add_BC[N2];
    real<lower=0,upper=2> amp_scal;
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4]-phi_i_add_RC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]-phi_i_add_RC[i]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+amp_scal*Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4]-phi_i_add_RC[i])+amp_scal*Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]-phi_i_add_RC[i]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4]-phi_i_add_BC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]-phi_i_add_BC[i]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+amp_scal*Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4]-phi_i_add_BC[i])+amp_scal*Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]-phi_i_add_BC[i]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    for (n in 1:N1) {
        phi_i_add_RC[n] ~ von_mises(0, kappa);
    }
    for (n in 1:N2) {
        phi_i_add_BC[n] ~ von_mises(0, kappa);
    }
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
}
generated quantities{ 
    vector[N1] log_lik1;
    vector[N1] log_lik2;
    vector[N2] log_lik3;
    vector[N2] log_lik4;
    real mu1;
    real mu2;
    real mu3;
    real mu4;
    real r1;
    real r2;
    real r3;
    real r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    for ( i in 1:N1 ) {
        b1 = burstCry1*(AreaNormed_RC[i])^beta_v_Cry1;
        b2 = burstNr1d1*(AreaNormed_RC[i])^beta_v_Nr1d1;
        f1 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4]-phi_i_add_RC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]-phi_i_add_RC[i]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+amp_scal*Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4]-phi_i_add_RC[i])+amp_scal*Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]-phi_i_add_RC[i]));
        mu1 = b1*f1;
        mu2 = b2*f2;
        r1 = f1;
        r2 = f2;
        log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
        log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
    } 
    for ( i in 1:N2 ) {
        b3 = burstCry1*(AreaNormed_BC[i])^beta_v_Cry1;
        b4 = burstBmal1*(AreaNormed_BC[i])^beta_v_Bmal1;
        f3 = freq_scaleCry1*(Cry1_params[1]/2+amp_scal*Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4]-phi_i_add_BC[i])+amp_scal*Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]-phi_i_add_BC[i]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+amp_scal*Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4]-phi_i_add_BC[i])+amp_scal*Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]-phi_i_add_BC[i]));
        mu3 = b3*f3;
        mu4 = b4*f4;
        r3 = f3;
        r4 = f4;
        log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
        log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
    } 
    

}        

"""

#%%

In [13]:
posterior = pickle.load(open('model_2.pkl', 'rb'))
fit = pickle.load(open('fit_2.pkl', 'rb')) 

a = fit

In [14]:
# load parameters from model M2
# freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=0)
# freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=0)
# freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=0)
# beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=0)
# beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=0)
# beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=0)
# burstCry1 = np.mean(a['burstCry1'],axis=0)
# burstNr1d1 = np.mean(a['burstNr1d1'],axis=0)
# burstBmal1 = np.mean(a['burstBmal1'],axis=0)

freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=1).squeeze()
freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=1).squeeze()
freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=1).squeeze()
beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=1).squeeze()
beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=1).squeeze()
beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=1).squeeze()
burstCry1 = np.mean(a['burstCry1'],axis=1).squeeze()
burstNr1d1 = np.mean(a['burstNr1d1'],axis=1).squeeze()
burstBmal1 = np.mean(a['burstBmal1'],axis=1).squeeze()

In [15]:
dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'kappa' : 2,
    'beta_v_Cry1' : beta_v_Cry1,
    'beta_v_Nr1d1' : beta_v_Nr1d1,
    'beta_v_Bmal1' : beta_v_Bmal1,
    'freq_scaleCry1' : freq_scaleCry1,
    'freq_scaleNr1d1' : freq_scaleNr1d1, 
    'freq_scaleBmal1' : freq_scaleBmal1, 
    'burstCry1' : burstCry1,
    'burstNr1d1' : burstNr1d1,
    'burstBmal1' : burstBmal1,
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}
#%%

In [16]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=16, num_samples=2000)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan__82_v1mv/model_g75f6d45.stan', line 4, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan__82_v1mv/model_g75f6d45.stan', line 5, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan__82_v1mv/model_g75f6d45.stan', line 6, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format

Sampling:   9% (4500/48000)
Sampling:  10% (4600/48000)
Sampling:  10% (4700/48000)
Sampling:  10% (4800/48000)
Sampling:  10% (4900/48000)
Sampling:  10% (5000/48000)
Sampling:  11% (5100/48000)
Sampling:  11% (5200/48000)
Sampling:  11% (5300/48000)
Sampling:  11% (5400/48000)
Sampling:  11% (5500/48000)
Sampling:  12% (5600/48000)
Sampling:  12% (5700/48000)
Sampling:  12% (5800/48000)
Sampling:  12% (5900/48000)
Sampling:  12% (6000/48000)
Sampling:  13% (6100/48000)
Sampling:  13% (6200/48000)
Sampling:  13% (6300/48000)
Sampling:  13% (6400/48000)
Sampling:  14% (6500/48000)
Sampling:  14% (6600/48000)
Sampling:  14% (6700/48000)
Sampling:  14% (6800/48000)
Sampling:  14% (6900/48000)
Sampling:  15% (7000/48000)
Sampling:  15% (7100/48000)
Sampling:  15% (7200/48000)
Sampling:  15% (7300/48000)
Sampling:  15% (7400/48000)
Sampling:  16% (7500/48000)
Sampling:  16% (7600/48000)
Sampling:  16% (7700/48000)
Sampling:  16% (7800/48000)
Sampling:  16% (7900/48000)
Sampling:  17% (8000

Sampling:  44% (20900/48000)
Sampling:  44% (21000/48000)
Sampling:  44% (21100/48000)
Sampling:  44% (21200/48000)
Sampling:  44% (21300/48000)
Sampling:  45% (21400/48000)
Sampling:  45% (21500/48000)
Sampling:  45% (21600/48000)
Sampling:  45% (21700/48000)
Sampling:  45% (21800/48000)
Sampling:  46% (21900/48000)
Sampling:  46% (22000/48000)
Sampling:  46% (22100/48000)
Sampling:  46% (22200/48000)
Sampling:  46% (22300/48000)
Sampling:  47% (22400/48000)
Sampling:  47% (22500/48000)
Sampling:  47% (22600/48000)
Sampling:  47% (22700/48000)
Sampling:  48% (22800/48000)
Sampling:  48% (22900/48000)
Sampling:  48% (23000/48000)
Sampling:  48% (23100/48000)
Sampling:  48% (23200/48000)
Sampling:  49% (23300/48000)
Sampling:  49% (23400/48000)
Sampling:  49% (23500/48000)
Sampling:  49% (23600/48000)
Sampling:  49% (23700/48000)
Sampling:  50% (23800/48000)
Sampling:  50% (23900/48000)
Sampling:  50% (24000/48000)
Sampling:  50% (24100/48000)
Sampling:  50% (24200/48000)
Sampling:  51%

Sampling:  81% (38700/48000)
Sampling:  81% (38800/48000)
Sampling:  81% (38900/48000)
Sampling:  81% (39000/48000)
Sampling:  81% (39100/48000)
Sampling:  82% (39200/48000)
Sampling:  82% (39300/48000)
Sampling:  82% (39400/48000)
Sampling:  82% (39500/48000)
Sampling:  82% (39600/48000)
Sampling:  83% (39700/48000)
Sampling:  83% (39800/48000)
Sampling:  83% (39900/48000)
Sampling:  83% (40000/48000)
Sampling:  84% (40100/48000)
Sampling:  84% (40200/48000)
Sampling:  84% (40300/48000)
Sampling:  84% (40400/48000)
Sampling:  84% (40500/48000)
Sampling:  85% (40600/48000)
Sampling:  85% (40700/48000)
Sampling:  85% (40800/48000)
Sampling:  85% (40900/48000)
Sampling:  85% (41000/48000)
Sampling:  86% (41100/48000)
Sampling:  86% (41200/48000)
Sampling:  86% (41300/48000)
Sampling:  86% (41400/48000)
Sampling:  86% (41500/48000)
Sampling:  87% (41600/48000)
Sampling:  87% (41700/48000)
Sampling:  87% (41800/48000)
Sampling:  87% (41900/48000)
Sampling:  88% (42000/48000)
Sampling:  88%

In [17]:
with open('model_3.pkl', 'wb') as f:
    pickle.dump(posterior, f)
    
with open('fit_3.pkl', 'wb') as g:
    pickle.dump(fit, g)

In [4]:
#%%     

# MODEL4

model1 = """
data{
    int<lower=1> N1;
    int CountsNr1d1_RC[N1];
    int CountsCry1_RC[N1];    
    real jtime_RC[N1];
    real AreaNormed_RC[N1];
    int<lower=1> N2;
    int CountsBmal1_BC[N2];
    int CountsCry1_BC[N2];    
    real jtime_BC[N2];
    real AreaNormed_BC[N2];
    real<lower=0> beta_v_Cry1;
    real<lower=0> beta_v_Nr1d1;
    real<lower=0> beta_v_Bmal1;
    real<lower=0> freq_scaleCry1;
    real<lower=0> freq_scaleNr1d1; 
    real<lower=0> freq_scaleBmal1; 
    int <lower=1> N_f;
    real Nr1d1_params[N_f];
    real Cry1_params[N_f];
    real Bmal1_params[N_f];
    real w;
}
parameters{
    real<lower=0> burstCry1;
    real<lower=0> burstNr1d1;
    real<lower=0> burstBmal1;
    matrix[N1,2] eta_RC;
    matrix[N2,2] eta_BC;
    real<lower=0> stdevCry1;
    real<lower=0> stdevNr1d1; 
    real<lower=0> stdevBmal1; 
    cholesky_factor_corr[2] L_RC;
    cholesky_factor_corr[2] L_BC;
}
transformed parameters{
    matrix[2, 2] corr_RC;
    matrix[2, 2] cov_RC;
    vector[2] mu_vec_RC;
    vector[2] sigma_vec_RC;
    matrix[2, 2] corr_BC;
    matrix[2, 2] cov_BC;
    vector[2] mu_vec_BC;
    vector[2] sigma_vec_BC;
    corr_RC = L_RC*L_RC';
    corr_BC = L_BC*L_BC';
    mu_vec_RC[1] = burstCry1;
    mu_vec_RC[2] = burstNr1d1;
    mu_vec_BC[1] = burstCry1;
    mu_vec_BC[2] = burstBmal1;
    sigma_vec_RC[1] = stdevCry1;
    sigma_vec_RC[2] = stdevNr1d1;
    sigma_vec_BC[1] = stdevCry1;
    sigma_vec_BC[2] = stdevBmal1; 
    cov_RC = quad_form_diag(corr_RC, sigma_vec_RC);
    cov_BC = quad_form_diag(corr_BC, sigma_vec_BC);
}
model{
    vector[N1] mu1;
    vector[N1] mu2;
    vector[N2] mu3;
    vector[N2] mu4;
    vector[N1] r1;
    vector[N1] r2;
    vector[N2] r3;
    vector[N2] r4;
    real b1;
    real b2;
    real b3;
    real b4;
    real f1;
    real f2;
    real f3;
    real f4;
    vector[2] rescaled_eta;
    for ( i in 1:N1 ) {
        rescaled_eta = mu_vec_RC + diag_pre_multiply(sigma_vec_RC,L_RC)*(eta_RC[i,:]');
        b1 = exp(beta_v_Cry1*log(AreaNormed_RC[i])+rescaled_eta[1]);
        b2 = exp(beta_v_Nr1d1*log(AreaNormed_RC[i])+rescaled_eta[2]);
        f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
        f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
        mu1[i] = b1*f1;
        mu2[i] = b2*f2;
        r1[i] = f1;
        r2[i] = f2;
    } 
    for ( i in 1:N2 ) {
        rescaled_eta = mu_vec_BC + diag_pre_multiply(sigma_vec_BC,L_BC)*(eta_BC[i,:]');
        b3 = exp(beta_v_Cry1*log(AreaNormed_BC[i])+rescaled_eta[1]);
        b4 = exp(beta_v_Bmal1*log(AreaNormed_BC[i])+rescaled_eta[2]);
        f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
        f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
        mu3[i] = b3*f3;
        mu4[i] = b4*f4;
        r3[i] = f3;
        r4[i] = f4;
    } 
    CountsCry1_RC ~ neg_binomial_2( mu1 , r1 );
    CountsNr1d1_RC ~ neg_binomial_2( mu2 , r2 );
    CountsCry1_BC ~ neg_binomial_2( mu3 , r3 );
    CountsBmal1_BC ~ neg_binomial_2( mu4 , r4 );
    to_vector(eta_RC) ~ normal(0, 1);
    to_vector(eta_BC) ~ normal(0, 1);
    burstCry1 ~ normal(0, 100);
    burstNr1d1 ~ normal(0, 100);
    burstBmal1 ~ normal(0, 100);
    L_RC ~ lkj_corr_cholesky(4.0);
    L_BC ~ lkj_corr_cholesky(4.0);
    stdevCry1 ~ normal(0, 100);
    stdevNr1d1 ~ normal(0, 100);
    stdevBmal1 ~ normal(0, 100);
}
generated quantities{ 
vector[N1] log_lik1;
vector[N1] log_lik2;
vector[N2] log_lik3;
vector[N2] log_lik4;
real mu1;
real mu2;
real mu3;
real mu4;
real r1;
real r2;
real r3;
real r4;
real b1;
real b2;
real b3;
real b4;
real f1;
real f2;
real f3;
real f4;
vector[2] rescaled_eta;
for ( i in 1:N1 ) {
    rescaled_eta = mu_vec_RC + diag_pre_multiply(sigma_vec_RC,L_RC)*(eta_RC[i,:]');
    b1 = exp(beta_v_Cry1*log(AreaNormed_RC[i])+rescaled_eta[1]);
    b2 = exp(beta_v_Nr1d1*log(AreaNormed_RC[i])+rescaled_eta[2]);
    f1 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_RC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_RC[i]*w-Cry1_params[5]));
    f2 = freq_scaleNr1d1*(Nr1d1_params[1]/2+Nr1d1_params[2]*cos(jtime_RC[i]*w-Nr1d1_params[4])+Nr1d1_params[3]*cos(2*jtime_RC[i]*w-Nr1d1_params[5]));
    mu1 = b1*f1;
    mu2 = b2*f2;
    r1 = f1;
    r2 = f2;
    log_lik1[i] = neg_binomial_2_lpmf(CountsCry1_RC[i] | mu1 , r1);
    log_lik2[i] = neg_binomial_2_lpmf(CountsNr1d1_RC[i] | mu2 , r2);
} 
for ( i in 1:N2 ) {
    rescaled_eta = mu_vec_BC + diag_pre_multiply(sigma_vec_BC,L_BC)*(eta_BC[i,:]');
    b3 = exp(beta_v_Cry1*log(AreaNormed_BC[i])+rescaled_eta[1]);
    b4 = exp(beta_v_Bmal1*log(AreaNormed_BC[i])+rescaled_eta[2]);
    f3 = freq_scaleCry1*(Cry1_params[1]/2+Cry1_params[2]*cos(jtime_BC[i]*w-Cry1_params[4])+Cry1_params[3]*cos(2*jtime_BC[i]*w-Cry1_params[5]));
    f4 = freq_scaleBmal1*(Bmal1_params[1]/2+Bmal1_params[2]*cos(jtime_BC[i]*w-Bmal1_params[4])+Bmal1_params[3]*cos(2*jtime_BC[i]*w-Bmal1_params[5]));
    mu3 = b3*f3;
    mu4 = b4*f4;
    r3 = f3;
    r4 = f4;
    log_lik3[i] = neg_binomial_2_lpmf(CountsCry1_BC[i] | mu3 , r3);
    log_lik4[i] = neg_binomial_2_lpmf(CountsBmal1_BC[i] | mu4 , r4);
} 


}       

"""

In [5]:
# load parameters from model M2

posterior = pickle.load(open('model_2.pkl', 'rb'))
fit = pickle.load(open('fit_2.pkl', 'rb')) 

a = fit    

# freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=0)
# freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=0)
# freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=0)
# beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=0)
# beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=0)
# beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=0)
freq_scaleCry1 = np.mean(a['freq_scaleCry1'],axis=1).squeeze()
freq_scaleNr1d1 = np.mean(a['freq_scaleNr1d1'],axis=1).squeeze()
freq_scaleBmal1 = np.mean(a['freq_scaleBmal1'],axis=1).squeeze()
beta_v_Cry1 = np.mean(a['beta_v_Cry1'],axis=1).squeeze()
beta_v_Nr1d1 = np.mean(a['beta_v_Nr1d1'],axis=1).squeeze()
beta_v_Bmal1 = np.mean(a['beta_v_Bmal1'],axis=1).squeeze()

In [8]:
dat = {
    'N1' : len(CountsNr1d1_RC),
    'CountsNr1d1_RC' : CountsNr1d1_RC,
    'CountsCry1_RC' : CountsCry1_RC,   
    'jtime_RC' : jtime_RC, 
    'AreaNormed_RC' : AreaNormed_RC, 
    'N2' : len(CountsBmal1_BC),
    'CountsBmal1_BC' : CountsBmal1_BC,
    'CountsCry1_BC' : CountsCry1_BC,    
    'jtime_BC' : jtime_BC,
    'AreaNormed_BC' : AreaNormed_BC,
    'beta_v_Cry1' : beta_v_Cry1,
    'beta_v_Nr1d1' : beta_v_Nr1d1,
    'beta_v_Bmal1' : beta_v_Bmal1,
    'freq_scaleCry1' : freq_scaleCry1,
    'freq_scaleNr1d1' : freq_scaleNr1d1, 
    'freq_scaleBmal1' : freq_scaleBmal1, 
    'N_f' : len(Nr1d1_params),
    'Nr1d1_params' : Nr1d1_params,
    'Cry1_params' : Cry1_params,
    'Bmal1_params' : Bmal1_params,
    'w' : w
}

In [9]:
# sm = pystan.StanModel(model_code=model1)
posterior = stan.build(model1, data=dat, random_seed=194838)

# fit = sm.sampling(data=dat, seed=194838, iter=2000, chains=4, control=dict(adapt_delta=0.95))
fit = posterior.sample(num_chains=16, num_samples=2000)

Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_z8pxzuya/model_dbwr4zum.stan', line 4, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_z8pxzuya/model_dbwr4zum.stan', line 5, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/tmp/httpstan_z8pxzuya/model_dbwr4zum.stan', line 6, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.33.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format

Sampling:  12% (5708/48000)
Sampling:  12% (5809/48000)
Sampling:  12% (5909/48000)
Sampling:  13% (6009/48000)
Sampling:  13% (6109/48000)
Sampling:  13% (6209/48000)
Sampling:  13% (6309/48000)
Sampling:  13% (6409/48000)
Sampling:  14% (6509/48000)
Sampling:  14% (6609/48000)
Sampling:  14% (6709/48000)
Sampling:  14% (6809/48000)
Sampling:  14% (6909/48000)
Sampling:  15% (7009/48000)
Sampling:  15% (7110/48000)
Sampling:  15% (7211/48000)
Sampling:  15% (7310/48000)
Sampling:  15% (7411/48000)
Sampling:  16% (7512/48000)
Sampling:  16% (7612/48000)
Sampling:  16% (7711/48000)
Sampling:  16% (7810/48000)
Sampling:  16% (7911/48000)
Sampling:  17% (8010/48000)
Sampling:  17% (8111/48000)
Sampling:  17% (8210/48000)
Sampling:  17% (8309/48000)
Sampling:  18% (8408/48000)
Sampling:  18% (8508/48000)
Sampling:  18% (8607/48000)
Sampling:  18% (8706/48000)
Sampling:  18% (8806/48000)
Sampling:  19% (8905/48000)
Sampling:  19% (9004/48000)
Sampling:  19% (9103/48000)
Sampling:  19% (9203

Sampling:  49% (23701/48000)
Sampling:  50% (23801/48000)
Sampling:  50% (23900/48000)
Sampling:  50% (24000/48000)
Sampling:  50% (24100/48000)
Sampling:  50% (24200/48000)
Sampling:  51% (24300/48000)
Sampling:  51% (24400/48000)
Sampling:  51% (24500/48000)
Sampling:  51% (24600/48000)
Sampling:  51% (24700/48000)
Sampling:  52% (24800/48000)
Sampling:  52% (24900/48000)
Sampling:  52% (25000/48000)
Sampling:  52% (25100/48000)
Sampling:  52% (25200/48000)
Sampling:  53% (25300/48000)
Sampling:  53% (25400/48000)
Sampling:  53% (25500/48000)
Sampling:  53% (25600/48000)
Sampling:  54% (25700/48000)
Sampling:  54% (25800/48000)
Sampling:  54% (25900/48000)
Sampling:  54% (26000/48000)
Sampling:  54% (26100/48000)
Sampling:  55% (26200/48000)
Sampling:  55% (26300/48000)
Sampling:  55% (26400/48000)
Sampling:  55% (26500/48000)
Sampling:  55% (26600/48000)
Sampling:  56% (26700/48000)
Sampling:  56% (26800/48000)
Sampling:  56% (26900/48000)
Sampling:  56% (27000/48000)
Sampling:  56%

Sampling:  86% (41500/48000)
Sampling:  87% (41600/48000)
Sampling:  87% (41700/48000)
Sampling:  87% (41800/48000)
Sampling:  87% (41900/48000)
Sampling:  88% (42000/48000)
Sampling:  88% (42100/48000)
Sampling:  88% (42200/48000)
Sampling:  88% (42300/48000)
Sampling:  88% (42400/48000)
Sampling:  89% (42500/48000)
Sampling:  89% (42600/48000)
Sampling:  89% (42700/48000)
Sampling:  89% (42800/48000)
Sampling:  89% (42900/48000)
Sampling:  90% (43000/48000)
Sampling:  90% (43100/48000)
Sampling:  90% (43200/48000)
Sampling:  90% (43300/48000)
Sampling:  90% (43400/48000)
Sampling:  91% (43500/48000)
Sampling:  91% (43600/48000)
Sampling:  91% (43700/48000)
Sampling:  91% (43800/48000)
Sampling:  91% (43900/48000)
Sampling:  92% (44000/48000)
Sampling:  92% (44100/48000)
Sampling:  92% (44200/48000)
Sampling:  92% (44300/48000)
Sampling:  92% (44400/48000)
Sampling:  93% (44500/48000)
Sampling:  93% (44600/48000)
Sampling:  93% (44700/48000)
Sampling:  93% (44800/48000)
Sampling:  94%

  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  b

...

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpsta

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained var

  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 100, column 4 to column 48)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpsta

  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: neg_binomial_2_lpmf: Location parameter[1] is inf, but must be positive finite! (in '/tmp/httpstan_vxhl5vu1/model_dbwr4zum.stan', line 99, column 4 to column 47)
  If this warning occurs sporadically, such as for highly constrained var

In [10]:
with open('model_4.pkl', 'wb') as f:
    pickle.dump(posterior, f)    

In [ ]:
with open('fit_4.pkl', 'wb') as g:
    pickle.dump(fit, g)   
    
# a = fit.extract(permuted=True)